In [56]:
import torch
import numpy as np
from mindspore import Tensor, dtype
from mindspore import save_checkpoint
from mindspore import Parameter

name_pth = "swinv2_base_patch4_window12_192_22k"
model = torch.load(f"{name_pth}.pth", map_location='cpu')['model']
for key in model:
    if "relative_coords_table" in key:
        print(key, model[key].shape)

layers.0.blocks.0.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.0.blocks.1.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.1.blocks.0.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.1.blocks.1.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.0.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.1.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.2.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.3.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.4.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.5.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.6.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.7.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.8.attn.relative_coords_table torch.Size([1, 23, 23, 2])
layers.2.blocks.9.attn.relative_coords_table torch.Size([1, 23, 

In [57]:
weights = []
prefix = "model."
for key in model.keys():
    name2weight = {}
    if "relative_position_index" in key or "attn_mask" in key:
        pass
    elif "norm" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        else:
            raise ValueError("Wrong")
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weights.append(name2weight)
    elif "qkv" in key:
        key_q = prefix + key.replace("qkv", "q")
        key_k = prefix + key.replace("qkv", "k")
        key_v = prefix + key.replace("qkv", "v")
        shape = model[key].shape[0]//3
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weight_q = weight[:shape]
        weight_k = weight[shape:shape*2]
        weight_v = weight[shape*2:]
        weights.append({"name":key_q, "data": weight_q})
        weights.append({"name":key_k, "data": weight_k})
        weights.append({"name":key_v, "data": weight_v})
    elif "q_bias" in key:
        k = prefix + key.replace("q_bias", "q.bias")
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True)
        weights.append({"name":k, "data": weight})
    elif "v_bias" in key:
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True)
        k = prefix + key.replace("v_bias", "v.bias")
        weights.append({"name":k, "data": weight})
    elif "relative_coords_table" in key:
        if 'layers.3' in key:
            k = prefix + key
            weight = torch.nn.functional.interpolate(model[key].permute(0,3,1,2), size=(15, 15), mode='bicubic').permute(0,2,3,1)
            weight = Parameter(Tensor(weight.numpy(), dtype.float32),requires_grad=True)
        else:
            k = prefix + key
            weight = torch.nn.functional.interpolate(model[key].permute(0,3,1,2), size=(31, 31), mode='bicubic').permute(0,2,3,1)
            weight = Parameter(Tensor(weight.numpy(), dtype.float32),requires_grad=True)
        # print(weight.shape)
        weights.append({"name":k, "data": weight})
    elif "head" in key:
        with open("image22kto1k.txt", "r") as f:
            lines = f.readlines()
        index = np.array([int(a) for a in lines])
        if "22k" in name:
            assert model[key].shape[0] != 0
        if model[key].shape[0] != 1000:
            weight = model[key].numpy()[index]
        else:
            weight = model[key].numpy()
        weight = Parameter(Tensor(weight, dtype.float32),requires_grad=True)
        weights.append({"name": prefix+key, "data": weight})
    else:
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32 if "index" not in key else dtype.int32),requires_grad=True)
        key = prefix + key
        weights.append({"name": key, "data": weight})

In [58]:
sum([np.prod(weights[i]['data'].shape) for i in range(len(weights))]), len(weights)

(87962000, 497)

In [59]:
save_checkpoint(weights, f"{name_pth}.ckpt")